In [57]:
# Сперва загружаю и настраиваю исходный датасет

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import scipy as sp
import pandas as pd
import sklearn

ds_class = pd.read_csv("csgo_class.csv")
#ds_reg = pd.read_csv("csgo_reg.csv")

# в них есть непонятный столбец Unnamed. Удаляем его
ds_class = ds_class.drop('Unnamed: 0', axis=1)
#ds_reg = ds_reg.drop('Unnamed: 0', axis=1)

# убираю все строки, где игроки 5 на 5
# P.s. предсказывать победу в начале раунда, конечно, интересно, но это очень сложно, и выше 78% у меня вообще никак не получилось добиться.
# Я смотрю время до конца раунда (в секундах), счет, деньги, живые игроки, их здоровье, бомба, дифуза, и самые популярные оружия (эмки, калаш, авп, ауг, сг)

ds_class = ds_class.loc[((ds_class['ct_players_alive'] != 5.0) & (ds_class['t_players_alive'] != 5.0))]


# преобразовываем нечисловые данные в числовые
ds_class_num = pd.get_dummies(ds_class)


# не преобразовался bool bomb_planted. Приводим в тип int
ds_class_num["bomb_planted"] = ds_class_num["bomb_planted"].astype(int)


import sklearn.preprocessing as preprocessing

# Данный нормализатор выравнивает данные относительно максимального по модулю
ma_scaler = preprocessing.MaxAbsScaler()
ma_scaler.fit(ds_class_num)
ma_norm_data = pd.DataFrame(columns=ds_class_num.columns, data=ma_scaler.transform(ds_class_num))

# переопределение переменных
# В перепенной X_train хранятся данные для тренировки сети (входные (время, деньги, живые игроки и т.д.))
# В перепенной Y_train хранятся данные для тренировки сети (выходные или правильные ответы (кто победил))
from sklearn.model_selection import train_test_split

X = ma_norm_data[ma_norm_data.columns[:-2]]
Y = ma_norm_data[ma_norm_data.columns[-1:]]

# Делим
# Делить будем в отношении 80/20
# 80 - это обучение
# 20 - тест
# первым делом перемешаем выборку (надо для того что бы обе выборки были репрезентативными)
XY_data = np.hstack( (X, Y) ).transpose((0,1))
np.random.shuffle(XY_data)
# Делим выборки
point_split = round(XY_data.shape[0] * 0.8)
train_data = XY_data[:point_split]
test_data = XY_data[point_split:]
print(f'Всего данных: {XY_data.shape[0]}')
print(f'Тренировочных данных: {train_data.shape[0]}')
print(f'Тестовых данных: {test_data.shape[0]}')

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=0.8)
X_test
# в Y террористы, т.е. предсказываю победу или поражение террористов
# 1 - победа, 0 - поражение

Всего данных: 26133
Тренировочных данных: 20906
Тестовых данных: 5227


,time_left,t_score,ct_score,t_money,ct_money,ct_weapon_ak47,t_weapon_ak47,ct_weapon_awp,t_weapon_awp,ct_weapon_m4a4,...,ct_weapon_aug,t_weapon_aug,ct_weapon_sg553,t_weapon_sg553,ct_defuse_kits,ct_players_alive,t_players_alive,ct_health,t_health,bomb_planted
5878,0.198571,0.090909,0.18750,0.010733,0.048480,0.25,0.50,0.00,0.0,0.00,...,0.25,0.0,0.00,0.00,0.25,0.50,0.50,0.4325,0.3550,1.0
17168,0.161657,0.333333,0.21875,0.161002,0.000822,0.00,0.50,0.00,0.5,0.25,...,0.00,0.0,0.00,0.00,0.00,0.25,1.00,0.1675,0.9750,1.0
24404,0.428286,0.454545,0.53125,0.078712,0.406738,0.00,0.25,0.25,0.0,0.25,...,0.00,0.0,0.25,0.25,0.75,0.75,0.50,0.5300,0.2300,0.0
15646,0.326914,0.333333,0.40625,0.007156,0.013147,0.50,0.25,0.50,0.0,0.00,...,0.00,0.0,0.00,0.00,0.25,1.00,0.25,0.8900,0.2450,0.0
22322,0.064457,0.333333,0.31250,0.067979,0.024651,0.00,0.25,0.00,0.0,0.75,...,0.00,0.0,0.00,0.00,0.00,0.75,0.25,0.7500,0.0675,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11202,0.342000,0.393939,0.46875,0.019678,0.110107,0.00,0.25,0.25,0.0,0.00,...,0.00,0.0,0.25,0.25,0.50,0.50,0.75,0.5000,0.5275,0.0
14163,0.200743,0.242424,0.40625,0.038462,0.048480,0.00,0.50,0.50,0.0,0.00,...,0.00,0.0,0.00,0.25,0.50,0.50,0.75,0.4925,0.6550,1.0
25995,0.124000,0.030303,0.03125,0.000894,0.009039,0.25,0.25,0.00,0.0,0.25,...,0.00,0.0,0.00,0.00,0.00,0.50,0.25,0.4275,0.2500,1.0
2293,0.428343,0.151515,0.12500,0.262970,0.095316,0.00,0.50,0.50,0.0,0.50,...,0.00,0.0,0.00,0.25,0.75,1.00,1.00,0.8350,0.9450,0.0


In [58]:
Y_test

,round_winner_T
5878,1.0
17168,1.0
24404,0.0
15646,0.0
22322,0.0
...,...
11202,1.0
14163,1.0
25995,0.0
2293,1.0


In [59]:
# Загружаю нейронку

from keras.models import load_model

# Загружаем контрольную точку (модель)
loaded_model = load_model('./save_points/nn_model_loss-0.35_val_loss-0.36_acc-0.83_val_acc-0.83.hdf5')

In [60]:
# Получаю точность

print(f'Точность предсказания на тестовых данных {loaded_model.evaluate(X_test, Y_test)[1]}')

164/164 [==============================] - 0s 994us/step - loss: 0.3479 - acc: 0.8248
Точность предсказания на тестовых данных 0.8247560858726501


In [61]:
# Средняя абсолютная ошибка (средняя разница между истинным значением и предсказанным (по модулю))

import tensorflow as tf

mae = tf.keras.losses.MeanAbsoluteError()
mae(Y_test, loaded_model.predict(X_test)).numpy()

164/164 [==============================] - 0s 890us/step


0.23074451

In [62]:
X_new = X_test.copy(deep=True)
errors = []
errors_cl = []

for column in X_test.columns: # для каждого столбца (признака)
    del X_new
    X_new = X_test.copy(deep=True)
    X_new[column] = 0 # зануляю его значения
    
    print(f'Точность без {column}: {loaded_model.evaluate(X_new, Y_test)[1]}')
    
    mae = tf.keras.losses.MeanAbsoluteError()
    m = mae(Y_test, loaded_model.predict(X_new)).numpy()
    print(f'Ошибка без {column}: {m}')
    print("--------------------------------------------")
    errors.append(m)
    errors_cl.append({column: m})
   

164/164 [==============================] - 0s 1ms/step - loss: 0.5456 - acc: 0.7850
Точность без time_left: 0.7849627137184143
164/164 [==============================] - 0s 908us/step
Ошибка без time_left: 0.24675512313842773
--------------------------------------------
164/164 [==============================] - 0s 1ms/step - loss: 0.3528 - acc: 0.8276
Точность без t_score: 0.8276258111000061
164/164 [==============================] - 0s 988us/step
Ошибка без t_score: 0.2243361622095108
--------------------------------------------
164/164 [==============================] - 0s 1ms/step - loss: 0.3568 - acc: 0.8244
Точность без ct_score: 0.8243734240531921
164/164 [==============================] - 0s 963us/step
Ошибка без ct_score: 0.23368783295154572
--------------------------------------------
164/164 [==============================] - 0s 1ms/step - loss: 0.3508 - acc: 0.8248
Точность без t_money: 0.8247560858726501
164/164 [==============================] - 0s 1ms/step
Ошибка без t_m

In [63]:
# Какие результаты:
# без time_left потеряли 5% точности и на 2% ошибка больше
# без t_score результаты стали даже немного лучше (без ct_score +- такие же)
# без ct_players_alive потеряли 12% точности и на 6% ошибка больше
# без t_players_alive потеряли 13% точности и на 8% ошибка больше
# без ct_health потеряли 9% точности и на 3% ошибка больше
# без t_health потеряли 7% точности и на 3% ошибка больше
# без bomb_planted потеряли 2% точности и на 3% ошибка больше
# без других признаков результаты почти не отличаются от начальных

In [64]:
errors.sort()
errors_position = []

i = 1
for error in errors:
  
    for error_cl in errors_cl:
        for key, value in error_cl.items():
            if value == error:
                print(f'{i} {key}: {error}')
                errors_position.append({key: i})
    i = i + 1
    
# Чем позиция признака больше, тем он круче (полезнее)

1 ct_defuse_kits: 0.22343489527702332
2 t_score: 0.2243361622095108
3 t_money: 0.23063232004642487
4 t_weapon_m4a1s: 0.23093801736831665
5 t_weapon_aug: 0.23101939260959625
6 ct_money: 0.23104526102542877
7 ct_weapon_aug: 0.23186343908309937
8 t_weapon_m4a4: 0.2321421504020691
9 ct_weapon_m4a1s: 0.23264741897583008
10 t_weapon_awp: 0.23312869668006897
11 ct_score: 0.23368783295154572
12 ct_weapon_sg553: 0.2338186353445053
13 ct_weapon_awp: 0.23453658819198608
14 ct_weapon_ak47: 0.23506702482700348
15 t_weapon_sg553: 0.23806288838386536
16 ct_weapon_m4a4: 0.2399391382932663
17 t_weapon_ak47: 0.24379391968250275
18 time_left: 0.24675512313842773
19 ct_health: 0.25665897130966187
20 bomb_planted: 0.2585928440093994
21 t_health: 0.26473569869995117
22 ct_players_alive: 0.28431472182273865
23 t_players_alive: 0.3143954277038574


In [65]:
import keras
from keras.layers import Dense
from keras.models import Sequential
from keras import activations
from keras.optimizers import Adam, RMSprop
from keras.losses import categorical_crossentropy, binary_crossentropy
from keras.metrics import categorical_accuracy, binary_accuracy
from keras import callbacks

def make_nn(input_len, output_len):
    model = Sequential()
    model.add(Dense(30, activation=activations.relu, input_dim=input_len))
    model.add(Dense(30, activation=activations.relu))
    model.add(Dense(output_len, activation=activations.sigmoid))
    model.compile(Adam(), binary_crossentropy, metrics=['acc'])
    return model

j = 1
for _ in X_test.columns:
    print(f'Обучение: {j}')
    model = make_nn(X_train.shape[1], Y_train.shape[1])
    history = model.fit(
    x=X_train,
    y=Y_train,
    batch_size=5,
    epochs=5,
    verbose=1,
    validation_data=(X_test, Y_test),
    callbacks=[
        callbacks.History(),
        callbacks.EarlyStopping(
            # На основе какого значения будет приниматься решеня об остановке
            monitor='val_acc',
            # Указываем направления лучшего значения (min, max, auto) лучшим является если тестовая точноть будет максимальной
            mode='max',
            # Количество эпох в резельтате которых если значение не изменилось, то произвести остановку
            patience=2,
            # "Чуствительность" метода - изменения ниже данного значения дубут считатья 0 (изменения в отслеживаемом значении нету)
            min_delta=0.0001,
            )
        ]
    
    )
    j += 1
    
    X_new = X_test.copy(deep=True)
    errors = []
    errors_cl = []
    for column in X_test.columns: # для каждого столбца (признака)
        del X_new
        X_new = X_test.copy(deep=True)
        X_new[column] = 0 # зануляю его значения
        mae = tf.keras.losses.MeanAbsoluteError()
        m = mae(Y_test, model.predict(X_new)).numpy()
        errors.append(m)
        errors_cl.append({column: m})
        
    errors.sort()
    errors_position_new = []
    i = 1
    for error in errors:
        for error_cl in errors_cl:
            for key, value in error_cl.items():
                if value == error:
                    print(f'{i} {key}: {error}')
                    errors_position_new.append({key: i})
        i = i + 1
    print()
    
    for error in errors_position:
        for key, value in error.items():
            
            for error2 in errors_position_new:
                for key2, value2 in error2.items():
                    if key == key2:
                        if value != value2:
                            print(f'Эталонная позиция {key}: {value}. Сейчас: {value2}')
    print("--------")
    
    
    
    

Обучение: 1
Epoch 1/5
4182/4182 [==============================] - 6s 1ms/step - loss: 0.3952 - acc: 0.8079 - val_loss: 0.3731 - val_acc: 0.8160
Epoch 2/5
4182/4182 [==============================] - 6s 1ms/step - loss: 0.3713 - acc: 0.8209 - val_loss: 0.3703 - val_acc: 0.8204
Epoch 3/5
4182/4182 [==============================] - 5s 1ms/step - loss: 0.3675 - acc: 0.8225 - val_loss: 0.3720 - val_acc: 0.8139
Epoch 4/5
164/164 [==============================] - 0s 910us/step
1 ct_defuse_kits: 0.22786405682563782
2 t_weapon_awp: 0.22859960794448853
3 ct_money: 0.22865429520606995
4 ct_score: 0.2290494740009308
5 t_score: 0.2291208654642105
6 t_weapon_aug: 0.22940890491008759
7 t_weapon_m4a1s: 0.22949647903442383
8 ct_weapon_awp: 0.2296319305896759
9 ct_weapon_aug: 0.22973018884658813
10 t_weapon_m4a4: 0.22974629700183868
11 ct_weapon_m4a1s: 0.2298840582370758
12 ct_weapon_ak47: 0.23024903237819672
13 t_money: 0.230442076921463
14 ct_weapon_sg553: 0.23059093952178955
15 time_left: 0.230811